In [2]:
# 降噪

%matplotlib inline
import sys
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
import cv2
import numpy as np

origin = '/Users/zhangzhichao/workplace/traindata/captcha/3C4HX7.png'

def denoise(path):
    src = cv2.imread(path)
    gray_src = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    #cv2.imshow("input image", src)
    #cv2.imshow("gray image", gray_src)
    #cv2.waitKey(0)
    gray_src = cv2.bitwise_not(gray_src)
    #二值化
    binary_src = cv2.adaptiveThreshold(gray_src, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, -2)
    # cv2.namedWindow("result image", cv2.WINDOW_AUTOSIZE)
    # cv2.imshow("result image", binary_src)
    #cv2.waitKey(0)
    # 提取水平线    src.shape[1]得到src列数
    #hline = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1), (-1, -1))

    hline = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 4), (-1, -1)) #定义结构元素，卷积核
    # print('hline: %s' % hline)
    # 提取垂直线    src.shape[0]得到src行数
    vline = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 1), (-1, -1))
    #vline = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    # 这两步就是形态学的开操作——先腐蚀再膨胀
    # temp = cv2.erode(binary_src, hline)     #腐蚀
    # dst = cv2.dilate(temp, hline)           #膨胀
    # 开运算
    dst = cv2.morphologyEx(binary_src, cv2.MORPH_OPEN, hline)    #水平方向
    dst = cv2.morphologyEx(dst, cv2.MORPH_OPEN, vline)    #垂直方向
    #将二指图片的效果反转既黑色变白色，白色变黑色。 非操作
    dst = cv2.bitwise_not(dst)
    #     cv2.imshow("Final image", dst)
    cv2.imwrite(path, dst)

    
def resize_img(path, shape):
    img = Image.open(path)
    img.resize(shape).save(path)
    
denoise('/Users/zhangzhichao/workplace/traindata/captcha/333894.png')

In [3]:
# 分割图片
import os
from uuid import uuid4
from PIL import Image
import math

    

def get_filename(path):
    return os.path.basename(path).split('.')[0].split('_')[0]

def get_labels(filename):
    pass

def crop_image(img, start, end, filename):
    # left, upper, right, and lower
    img.crop((start, 0, end, img.size[1])).save(filename)
    
def split_image(path):
    filename = get_filename(path)
#     labels = filename.split()
    num_chars = len(filename)
    img = Image.open(path)
    width, height = img.size
    d = math.ceil(width/num_chars)
    
    data_dir = 'splited'
    for i, char in enumerate(filename):
        if not os.path.exists(data_dir):
            os.mkdir(data_dir)
#         print(i*d, (i+1)*d)
        crop_image(img, i*d, (i+1) * d, os.path.join(data_dir, '%s_%s.png' % (char, uuid4().hex)))

# denoise('3as47g.png')
# split_image('3as47g.png')

In [4]:
# 批量操作图片
import pathlib

def process_img(img):
    denoise(img)
#     split_image(img)

def bulk_split_images(dirpath):
    count = 0
    root_path = pathlib.Path(dirpath)
    for img in root_path.glob('*.png'):
        img = str(img)
        # 处理图片
        process_img(img)
        # 分割图片
        split_image(img)
        count += 1
    print('完成:%s' % count)

bulk_split_images('/Users/zhangzhichao/workplace/traindata/data')

完成:1081


In [5]:
# 随机分为train and validate
import pathlib
from random import sample
import os

def random_split_to_train_and_validate(path):
    root_path = pathlib.Path(path)
    all_images = list(root_path.glob('*.png'))
    total = len(all_images)
    num_validate = total // 3
    validate_images = sample(all_images, num_validate)

    validata_dir = 'test'
    train_dir = 'train'
    if not os.path.exists(validata_dir):
        os.mkdir(validata_dir)
    if not os.path.exists(train_dir):
        os.mkdir(train_dir)
    for img in validate_images:
        img = str(img)
        os.rename(img, '%s/%s' % (validata_dir, os.path.basename(img)))
    all_images = list(root_path.glob('*.png'))
    for img in all_images:
        img = str(img)
        os.rename(img, '%s/%s' % (train_dir, os.path.basename(img)))
        
random_split_to_train_and_validate('splited')